# Watchdogs
The Watchdog class is a modular monitoring unit which periodically runs one experiment and compares the result against a defined threshold. Thresholds are defined with a tuple of the lower and upper bound; passing None to either element will deactivate that bound.

In [1]:
from watchdog import Watchdog

def experiment():
    return 2

params = {}
w = Watchdog(experiment=experiment, threshold=(0, 1), name='MyWatchdog')
w.check()

(2, False)

The check() method returns a tuple consisting of the measured value and the result of the logical comparison to the specified thresholds.

When a check fails, the Watchdog calls its react() method. This method does nothing by default, but you can implement any custom functionality to respond to problems with monitored variables. For example, let's reimplement w.react() to print a big warning message when the check fails:

In [2]:
def warning():
    print('WARNING: Watchdog variable out of range!!')
    
w.react = warning
w.check()

(2, False)

Practical implementation of a Watchdog-based monitoring system in the lab will probably involve passing ADC read methods to the Watchdog.experiment attribute. For example, suppose the function read_ADC(ch) reads a channel labeled by the integer 'ch' (we'll return dummy values of 0 and 2 instead):

In [3]:
import numpy as np

def read_ADC(ch):
    mu = [0.5, 2][ch]
    sigma = [0.1, 0.5][ch]
    return np.random.normal(mu, sigma)

To monitor both channels, we would simply create two Watchdog objects, each pointing at a given channel:

In [4]:
from functools import partial, partialmethod
watchdogs = {}
for ch in range(2):
    func = partial(read_ADC, ch)
    watchdogs[str(ch)] = Watchdog(func, threshold=(0,1))

print(watchdogs['0'].check())
print(watchdogs['1'].check())

(0.39993110752489147, True)
(1.7695772876562152, False)


# Monitors
The Monitor class manages multiple Watchdogs. We'll use it to handle the two objects defined in the previous example. If a filename is passed to the Monitor constructor, the result will automatically be appended to the file in a new line.

In [ ]:
from watchdog import Monitor
        
m = Monitor(watchdogs, filename = 'watchdog_test.txt')
m.check()

For continuous periodic monitoring, call the Monitor.start_periodic() method with your desired period. For example, let's check once per second:

In [ ]:
m.start_periodic(1)

Note that the Monitor.start_periodic method uses Python's sched module for event scheduling to avoid accumulated drifts. For example, if your check() method takes 50 ms to return and you call it in a loop with a 1s delay, the true time between checks will actually be 1.05 s. Using the scheduler allows drift-free checking; however, you should make sure that the check() call always returns in a time shorter than your chosen period!

To stop the monitoring, just call the Monitor.stop() method:

In [ ]:
m.stop()

Monitoring can be synced to some other process through software triggers, e.g. TTL pulses acquired with a DAQ board. Just define a method which returns as soon as a trigger is received and pass it into the Monitor.start_triggered() method. Let's simulate a trigger arriving once per second:

In [ ]:
import time
def trigger():
    time.sleep(1)
    return

m.start_triggered(trigger)

In [ ]:
m.stop()

### Plotting
Call the Monitor.plot() method to display a live plot of all monitored channels:

In [ ]:
m.plot()

### Callbacks
You can pass a callback to the monitor to implement custom functionality with each monitoring cycle. For example, let's create a class which displays the last 5 datapoints in real time:

In [34]:
from watchdog import Monitor
import pandas as pd 
from IPython.display import display, HTML, clear_output
from ipywidgets import Output

class Display:
    def __init__(self):
        self.data = pd.DataFrame()
        self.output = Output()
        display(self.output)
        
    def callback(self, data):
        self.data = self.data.append(data)
        self.data = self.data[-5::]
        with self.output:
            clear_output(wait=True)
            display(HTML(self.data.to_html()))
    
d = Display()
m_cb = Monitor(watchdogs, filename = 'watchdog_test.txt', callback=d.callback, visualize=True)

m_cb.start_periodic(1)

Output()

In [36]:
m_cb.stop()

In [35]:
m_cb.plot()

FigureWidget({
    'data': [{'mode': 'markers',
              'name': '0',
              'type': 'scatter',
  …